# PDF to Text OCR Demo
## 使用Tesseract OCR批量处理arXiv PDF文件

这个notebook演示了如何使用Tesseract OCR将PDF文件转换为文本，并保持原有的布局结构。

### 1. 安装依赖库

首先安装所需的Python库：

In [ ]:
# 安装依赖库
!pip install -r requirements.txt

### 2. 导入必要的库

In [ ]:
import os
import sys
from pathlib import Path
import logging
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

# 导入OCR相关库
try:
    import pytesseract
    from pdf2image import convert_from_path
    from PIL import Image
    import cv2
    print("✓ 所有依赖库导入成功")
except ImportError as e:
    print(f"❌ 缺少依赖库: {e}")
    print("请运行: pip install -r requirements.txt")

### 3. 检查Tesseract OCR安装

In [ ]:
# 检查Tesseract是否已安装
try:
    version = pytesseract.get_tesseract_version()
    print(f"✓ Tesseract OCR版本: {version}")
except Exception as e:
    print(f"❌ Tesseract未找到: {e}")
    print("请安装Tesseract OCR:")
    print("  macOS: brew install tesseract")
    print("  Ubuntu: sudo apt install tesseract-ocr")
    print("  Windows: 下载并安装 https://github.com/UB-Mannheim/tesseract/wiki")

### 4. 导入OCR处理器类

In [ ]:
# 导入我们创建的OCR处理器
from pdf_ocr_batch import PDFOCRProcessor, download_arxiv_pdfs

print("✓ OCR处理器导入成功")

### 5. 下载示例PDF文件

In [ ]:
# 下载一些示例arXiv论文PDF
print("正在下载示例PDF文件...")
success = download_arxiv_pdfs(Path("pdfs"))

if success:
    print("✓ 示例PDF文件下载完成")
    
    # 列出下载的文件
    pdf_files = list(Path("pdfs").glob("*.pdf"))
    print(f"下载的PDF文件数量: {len(pdf_files)}")
    for pdf_file in pdf_files:
        size_mb = pdf_file.stat().st_size / (1024 * 1024)
        print(f"  - {pdf_file.name} ({size_mb:.1f} MB)")
else:
    print("❌ 下载失败，请检查网络连接")

### 6. 演示单个PDF页面的OCR处理

In [ ]:
# 选择第一个PDF文件进行演示
pdf_files = list(Path("pdfs").glob("*.pdf"))

if pdf_files:
    demo_pdf = pdf_files[0]
    print(f"演示文件: {demo_pdf.name}")
    
    # 转换PDF的第一页为图像
    print("正在转换PDF第一页为图像...")
    pages = convert_from_path(demo_pdf, dpi=200, first_page=1, last_page=1)
    
    if pages:
        first_page = pages[0]
        
        # 显示原始图像
        plt.figure(figsize=(12, 16))
        plt.subplot(1, 2, 1)
        plt.imshow(first_page)
        plt.title("原始PDF页面")
        plt.axis('off')
        
        # 创建OCR处理器并预处理图像
        processor = PDFOCRProcessor()
        processed_image = processor.preprocess_image(first_page)
        
        # 显示预处理后的图像
        plt.subplot(1, 2, 2)
        plt.imshow(processed_image, cmap='gray')
        plt.title("预处理后的图像")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"图像尺寸: {first_page.size}")
    else:
        print("❌ 无法转换PDF页面")
else:
    print("❌ 没有找到PDF文件")

### 7. 提取文本并显示结果

In [ ]:
# 从第一页提取文本
if 'first_page' in locals():
    print("正在进行OCR文本提取...")
    
    # 提取文本
    text = processor.extract_text_from_image(first_page, 1)
    
    print("\n" + "="*60)
    print("OCR提取的文本 (前500字符):")
    print("="*60)
    print(text[:500])
    
    if len(text) > 500:
        print("\n[文本已截断，完整文本会保存到输出文件中]")
    
    print(f"\n提取的文本总长度: {len(text)} 字符")
    print(f"文本行数: {len(text.splitlines())}")

### 8. 批量处理所有PDF文件

In [ ]:
# 创建OCR处理器
processor = PDFOCRProcessor(output_dir="pdf_ocr", dpi=300)

print("开始批量处理PDF文件...")
print("注意: 这可能需要几分钟时间，取决于PDF的大小和页数")

# 批量处理
results = processor.batch_process(Path("pdfs"))

# 显示处理结果
print("\n" + "="*60)
print("批量处理结果:")
print("="*60)

if "error" in results:
    print(f"❌ 处理失败: {results['error']}")
else:
    print(f"总文件数: {results['total']}")
    print(f"✓ 成功处理: {results['successful']}")
    print(f"❌ 处理失败: {results['failed']}")
    
    if results['failed_files']:
        print(f"失败文件: {', '.join(results['failed_files'])}")
    
    success_rate = (results['successful'] / results['total']) * 100 if results['total'] > 0 else 0
    print(f"成功率: {success_rate:.1f}%")

### 9. 检查输出文件

In [ ]:
# 列出生成的文本文件
output_files = list(Path("pdf_ocr").glob("*.txt"))

print(f"生成的文本文件数量: {len(output_files)}")
print("\n输出文件列表:")

for txt_file in output_files:
    size_kb = txt_file.stat().st_size / 1024
    print(f"  - {txt_file.name} ({size_kb:.1f} KB)")

# 显示第一个文件的开头内容
if output_files:
    first_txt = output_files[0]
    print(f"\n文件 '{first_txt.name}' 的开头内容:")
    print("-" * 50)
    
    with open(first_txt, 'r', encoding='utf-8') as f:
        content = f.read(800)  # 读取前800字符
        print(content)
        
    if len(content) >= 800:
        print("\n[内容已截断...]")

### 10. OCR质量分析

In [ ]:
# 分析OCR结果质量
if output_files:
    print("OCR质量分析:")
    print("=" * 40)
    
    total_chars = 0
    total_words = 0
    total_lines = 0
    
    for txt_file in output_files:
        with open(txt_file, 'r', encoding='utf-8') as f:
            content = f.read()
            chars = len(content)
            words = len(content.split())
            lines = len(content.splitlines())
            
            total_chars += chars
            total_words += words
            total_lines += lines
            
            print(f"{txt_file.name}:")
            print(f"  字符数: {chars:,}")
            print(f"  单词数: {words:,}")
            print(f"  行数: {lines:,}")
            print()
    
    print("总计:")
    print(f"  总字符数: {total_chars:,}")
    print(f"  总单词数: {total_words:,}")
    print(f"  总行数: {total_lines:,}")
    print(f"  平均每个文件字符数: {total_chars/len(output_files):,.0f}")

### 11. 使用说明

#### 命令行使用方法:

```bash
# 基本用法 - 处理pdfs/目录下的所有PDF文件
python pdf_ocr_batch.py

# 指定输入和输出目录
python pdf_ocr_batch.py -i /path/to/pdfs -o /path/to/output

# 下载示例文件并处理
python pdf_ocr_batch.py --download

# 调整DPI设置 (更高DPI = 更好质量但更慢)
python pdf_ocr_batch.py --dpi 600
```

#### 输出文件格式:
- 每个PDF文件生成一个对应的.txt文件
- 保持页面结构，包含页码分隔符
- 包含处理时间和页数信息
- 使用UTF-8编码保存

#### 配置选项:
- **DPI设置**: 影响图像质量和处理速度 (推荐: 300-600)
- **OCR语言**: 默认英文，可修改为其他语言
- **页面分割模式**: 当前使用PSM 6，适合大多数学术论文

#### 注意事项:
- 确保系统已安装Tesseract OCR
- PDF文件越大，处理时间越长
- 图像质量会影响OCR准确性
- 复杂布局（如多栏、表格）可能需要调整参数